<a href="https://colab.research.google.com/github/JoeyVillafuerte/Simple-DocParse/blob/main/Simple_DocParse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Keep in mind these are scored and highlighted with the objective of showing it exists, NOT to show how accurate the information is.**

Process for running the System Tester

Run the installations for the master of Haystack

Run the no check certificate

Run !tar for linux tools

Start Elasticsearch and wait until it is ready

Connect to ElasticSearch

Run the cell to delete the stored documents.

Click on the folder on the left side of the screen, and go into the "data" folder. Delete the files and whatever is inside of the directory there.
ie data/sample
Import the retriever

in the cell with lots of the zipped links, input the s3 url into s3_url. it will look like s3_url = "insertlinkhere.zip" it **MUST** be a zip file, containing only the txt file needed, NOT nested into another folder.

Make sure that the INFO - haystack.utils.import_utils -  Fetching from "___" fetches the correct zip file

Optionally, print the doc and ensure it's correct.

Run the reader to hug someone and get the reader

Get the pipelines

Get predictions for the current queries. Currently it is Pools, Pets, and Parking.

Running the final 4 cells will just show the exact info and wording of a section, if it exists. If it does not exist, the score will usually be low and show random info to try and fill the answer.

In [ ]:
# Install the latest master of Haystack
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]

In [ ]:
!wget --no-check-certificate https://dl.xpdfreader.com/xpdf-tools-linux-4.04.tar.gz

In [ ]:
!tar -xvf xpdf-tools-linux-4.04.tar.gz && sudo cp xpdf-tools-linux-4.04/bin64/pdftotext /usr/local/bin

In [4]:
# Start Elasticsearch in Colab from source
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT

es_server = Popen(
    ["elasticsearch-7.9.2/bin/elasticsearch"], stdout=PIPE, stderr=STDOUT, preexec_fn=lambda: os.setuid(1)  # as daemon
)
# wait until ES has started
! sleep 30

In [ ]:
# Connect to Elasticsearch

from haystack.document_stores import ElasticsearchDocumentStore

document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")

In [7]:
# cap, you need to run this
from haystack.utils import clean_wiki_text, convert_files_to_docs, fetch_archive_from_http, print_answers
from haystack.nodes import FARMReader, TransformersReader

In [8]:
# deletes documents
document_store.delete_documents () 

In [9]:
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store)

In [ ]:
# https://email-template-attachments.s3.amazonaws.com/haystacktest/cedar+falls+rules_00001.zip
# https://email-template-attachments.s3.amazonaws.com/haystacktest/legacy_00001.zip
# https://email-template-attachments.s3.amazonaws.com/haystacktest/Rules+%26+Regs+Updated+2021_00001.zip
# https://email-template-attachments.s3.amazonaws.com/haystacktest/Rules+%26+Regs++cathedrael_00001.zip
# https://email-template-attachments.s3.amazonaws.com/haystacktest/Community+Handbook+2021+cody+place_00001.zip
# https://email-template-attachments.s3.amazonaws.com/haystacktest/pool+for+sure.zip

# This one is zipped wrong but still testing with it
# https://email-template-attachments.s3.amazonaws.com/haystacktest/washington+1+txt+files.zip

doc_dir = "data/sample"
s3_url = "https://email-template-attachments.s3.amazonaws.com/haystacktest/cedar+falls+rules_00001.zip"
fetch_archive_from_http(url=s3_url, output_dir=doc_dir)

# Convert files to dicts
# You can optionally supply a cleaning function that is applied to each doc (e.g. to remove footers)
# It must take a str as input, and return a str.
docs = convert_files_to_docs(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)

# We now have a list of dictionaries that we can write to our document store.
# If your texts come from a different source (e.g. a DB), you can of course skip convert_files_to_dicts() and create the dictionaries yourself.
# The default format here is:
# {
#    'content': "<DOCUMENT_TEXT_HERE>",
#    'meta': {'name': "<DOCUMENT_NAME_HERE>", ...}
# }
# (Optionally: you can also add more key-value-pairs here, that will be indexed as fields in Elasticsearch and
# can be accessed later for filtering or shown in the responses of the Pipeline)

# Let's have a look at the first 3 entries:
print(docs[:3])

# Now, let's write the dicts containing documents to our DB.
document_store.write_documents(docs)

In [ ]:
print (docs)

In [ ]:
# Load a  local model or any of the QA models on
# Hugging Face's model hub (https://huggingface.co/models)

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

In [13]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

In [ ]:
# You can configure how many candidates the reader and retriever shall return
# The higher top_k_retriever, the better (but also the slower) your answers.
predictionPets = pipe.run(
    query="Are pets or animals allowed?", params={"Retriever": {"top_k": 20}, "Reader": {"top_k": 3}}
)
predictionPool = pipe.run(
    query="Is there a pool in my community?", params={"Retriever": {"top_k": 20}, "Reader": {"top_k": 3}}
)
predictionParking = pipe.run(
    query="Are there any parking rules?", params={"Retriever": {"top_k": 20}, "Reader": {"top_k": 3}}
)
predictionContactInfo = pipe.run(
    query="Who do I contact in this community?", params={"Retriever": {"top_k": 20}, "Reader": {"top_k": 3}}
)
predictionPortal = pipe.run(
    query="What should I know for the Resident Portal in this community?", params={"Retriever": {"top_k": 20}, "Reader": {"top_k": 3}}
)
predictionARC = pipe.run(
    query="What are the architectural or renovation rules for this community?", params={"Retriever": {"top_k": 20}, "Reader": {"top_k": 3}}
)
predictionEmergency = pipe.run(
    query="What should I do in an emergency? Who do I contact?", params={"Retriever": {"top_k": 20}, "Reader": {"top_k": 3}}
)

In [15]:
for ans in predictionPets["answers"]:
  print("predictionPets Answer: " + '\033[1;30;42m ' + str(ans.answer) + ' \033[0;0m')
  print("Context: " + str(ans.context))
  print("Score: " + str(round(ans.score * 100,2)) + " %\n")

print("\n")  
for ans in predictionPool["answers"]:
  print("predictionPool Answer: " + '\033[1;30;42m ' + str(ans.answer) + ' \033[0;0m')
  print("Context: " + str(ans.context))
  print("Score: " + str(round(ans.score * 100,2)) + " %\n")

print("\n") 
for ans in predictionParking["answers"]:
  print("prediction Parking Answer: " + '\033[1;30;42m ' + str(ans.answer) + ' \033[0;0m')
  print("Context: " + str(ans.context))
  print("Score: " + str(round(ans.score * 100,2)) + " %\n")

print("\n") 
for ans in predictionContactInfo["answers"]:
  print("predictionContactInfo Answer: " + '\033[1;30;42m ' + str(ans.answer) + ' \033[0;0m')
  print("Context: " + str(ans.context))
  print("Score: " + str(round(ans.score * 100,2)) + " %\n")

print("\n") 
for ans in predictionPortal["answers"]:
  print("predictionPortal Answer: " + '\033[1;30;42m ' + str(ans.answer) + ' \033[0;0m')
  print("Context: " + str(ans.context))
  print("Score: " + str(round(ans.score * 100,2)) + " %\n")

print("\n") 
for ans in predictionARC["answers"]:
  print("predictionARC Answer: " + '\033[1;30;42m ' + str(ans.answer) + ' \033[0;0m')
  print("Context: " + str(ans.context))
  print("Score: " + str(round(ans.score * 100,2)) + " %\n")

print("\n") 
for ans in predictionEmergency["answers"]:
  print("predictionEmergency Answer: " + '\033[1;30;42m ' + str(ans.answer) + ' \033[0;0m')
  print("Context: " + str(ans.context))
  print("Score: " + str(round(ans.score * 100,2)) + " %\n")

predictionPets Answer:  Well-behaved domestic pets 
Context: mpact from pet defecation
must not be a problem to neighbors.
Well-behaved domestic pets are allowed, however, pet owners are
responsible for their pe
Score: 56.61 %

predictionPets Answer:  other animals are permitted to remain 
Context:  exercise the authority for specific animals even
though other animals are permitted to remain.
Swimming Pools, Above Ground--Portable or Permanent ar
Score: 50.81 %

predictionPets Answer:  Incessant barking of dogs is not permitted 
Context: by threatening or assaulting other residents or their
Incessant barking of dogs is not permitted by these rules and it also
violates King County Code 
Score: 49.53 %



predictionPool Answer:  Swimming Pools 
Context:  the Board and
ARTICLE 7.0 ADDITIONAL RESTRICTIONS AND PROHIBITIONS
Swimming Pools, Above Ground--Portable or Permanent
Carport, Garage Additions PROH
Score: 79.11 %

predictionPool Answer:  Swimming Pools 
Context: specific animals even


In [ ]:
# Now you can either print the object directly...
from pprint import pprint

pprint(predictionPool)
pprint(predictionPets)
pprint(predictionParking)

In [ ]:
# PETS TIME
# Change `minimum` to `medium` or `all` to raise the level of detail
print_answers(predictionPets, details="minimum")

In [ ]:
# POOL TIME
# Change `minimum` to `medium` or `all` to raise the level of detail
print_answers(predictionPool, details="minimum")

In [ ]:
# PARKING TIME
# Change `minimum` to `medium` or `all` to raise the level of detail
print_answers(predictionParking, details="minimum")